# ShorelineMonitor: Satellite-Derived Shoreline-Series

The
[ShorelineMonitor-Shorelines](https://radiantearth.github.io/stac-browser/#/external/coclico.blob.core.windows.net/stac/v1/shorelinemonitor-shorelines/collection.json)
dataset provides Satellite-Derived Shorelines (SDS) extracted from annually composited
Landsat satellite imagery spanning the years 1984-2024. These shorelines are mapped onto
the [Global Coastal Transect System
(GCTS)](https://github.com/TUDelft-CITG/coastpy/blob/main/tutorials/global_coastal_transect_system.ipynb).
Together they compose a new dataset that consists of time series per transect.
The ShorelineMonitor-Series consists of more than 350 million observations, each with 54
attributes, accross almost 7.5 million transects. The dataset and attributes are described in this [STAC
collection](https://radiantearth.github.io/stac-browser/#/external/coclico.blob.core.windows.net/stac/v1/shorelinemonitor-series/collection.json).
Please have a look at the metadata in one of the items. The dataset is available upon reasonable request. Please contact the data provider for more information or collaboration opportunities.  

In [ ]:
import os

import dotenv
import fsspec
import geopandas as gpd
import hvplot.pandas
import pandas as pd
import pystac
import shapely
from dotenv import load_dotenv
from ipyleaflet import Map, basemaps

from coastpy.stac.utils import read_snapshot

load_dotenv()

# Configure cloud and Dask settings
sas_token = os.getenv("AZURE_STORAGE_SAS_TOKEN")
storage_options = {"account_name": "coclico", "sas_token": sas_token}

coclico_catalog = pystac.Catalog.from_file(
    "https://coclico.blob.core.windows.net/stac/v1/catalog.json"
)
collection = coclico_catalog.get_child("shorelinemonitor-series")

In [ ]:
snapshot = read_snapshot(collection, storage_options=storage_options)
snapshot.head()

## Spatial partitions

This map shows the spatial partitions. In this release we have prioritized having unique transect_ids per partition over geospatial sorting. As a result there are overlapping partitions. This can be improved in the future. 

In [ ]:
snapshot.hvplot(geo=True, tiles="OSM", line_color="black", fill_alpha=0.2)

## Select a region of interest

Wait for the map to render and zoom to area of interest. 

In [ ]:
from ipyleaflet import Map, basemaps

m = Map(basemap=basemaps.Esri.WorldImagery, scroll_wheel_zoom=True)
m.center = (43.32, -1.97)
m.zoom = 14
m.layout.height = "800px"
m

In [ ]:
from coastpy.geo.utils import get_region_of_interest_from_map

roi = get_region_of_interest_from_map(m, default_extent=(4.796, 53.108, 5.229, 53.272))
west, south, east, north = list(roi.total_bounds)

## Fetch data

In [ ]:
import coastpy

db = coastpy.io.STACQueryEngine(
    stac_collection=collection,
    storage_backend="azure",
    # columns = ["geometry", "transect_id", "sds:change_rate"] ... # when you don't need all data
)

In [ ]:
from coastpy.utils.config import fetch_sas_token

sas_token = fetch_sas_token(sas_token)
df = db.get_data_within_bbox(west, south, east, north, sas_token=sas_token)
print(f"Shape: {df.shape}")
df.head()

## Explore data in a dashboard

In [ ]:
from coastpy.viz.dashboard import ShorelineSeriesApp

ShorelineSeriesApp(df).show()

## Process the dataset in bulk

In [ ]:
import dask_geopandas

fs = fsspec.filesystem("az", **storage_options)

urlpaths = series_snapshot.href.to_list()

ddf = dask_geopandas.read_parquet(urlpaths, filesystem=fs)
ddf

In [ ]:
def process(df):
    # your code comes here
    ...
    return df


result = ddf.map_partitions(process)